# ** Env Set Up ** #

In [1]:
# Import libraries
import os
import cv2
import time
import random
import gc as G
import numpy as np
import pandas as pd
import tensorflow as tf
from pathlib import Path
from numba import cuda
from tensorflow import keras
from tensorflow.keras import backend as K, mixed_precision
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from concurrent.futures import ThreadPoolExecutor
import keras_tuner as kt
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from keras_tuner import BayesianOptimization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau as RLOP, ModelCheckpoint
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier as KC
from sklearn.model_selection import RandomizedSearchCV as RSCV
from utils import view_train_images, plot_training_results, fast_import2 as FI2, get_images
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, GlobalAveragePooling2D, BatchNormalization, Flatten


In [2]:
# limit usage of GPU memory
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)  # Prevents TensorFlow from allocating all memory at once
            print(f"Enabled memory growth for: {gpu}")

        # Set GPU device
        device = "/GPU:0"
    except RuntimeError as e:
        print(e)
else:
    device = "/CPU:0"
    print("No GPU detected, using CPU.")

print(f"Using device: {device}")

# set gpu to use max memory
if gpus:
    try:
        for gpu in gpus:
            tf.config.set_logical_device_configuration(
                gpu,
                [tf.config.LogicalDeviceConfiguration(memory_limit=3072)]  # Set in MB (e.g., 10GB)
            )
        print("Set TensorFlow to use max GPU memory.")
    except RuntimeError as e:
        print(e)


Enabled memory growth for: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
Using device: /GPU:0
Set TensorFlow to use max GPU memory.


# 1) Data Import #

In [3]:
# paths
train_dir = "C:/Users/RoiMinuit/Desktop/data/ILSVRC/Data/CLS-LOC/small_train"
# val_dir = "C:/Users/RoiMinuit/Desktop/data/ILSVRC/Data/CLS-LOC/val"
# test_dir = "C:/Users/RoiMinuit/Desktop/data/ILSVRC/Data/CLS-LOC/test"

# path to save tuner results
kt_path = "C:/Users/RoiMinuit/Desktop/data/ILSVRC/Tuning_Res/KT"

# save best model
model_path = "C:/Users/RoiMinuit/Desktop/data/ILSVRC/Tuning_Res/KT/Model"


In [4]:
# data import
train_images, train_labels, val_images, val_labels = get_images(train_dir, train_dir, 8000)

Function processed 8000 images in 5 seconds.

Function processed 1200 images in 1 seconds.



In [5]:
# prepare training data
X_train = np.array(train_images)
X_train = np.repeat(X_train, 3, -1)

Y_train = [str(s) for s in train_labels]
label_encoder = LabelEncoder()
Y_train = label_encoder.fit_transform(Y_train)
Y_hot = to_categorical(Y_train)

# check data shape
# print(Y_train.shape)
# print(Y_val.shape)

# print(len(np.unique(Y_train)))
# print(len(np.unique(Y_val)))

In [6]:
# # prepare testing data
X_val = np.array(val_images)
X_val = np.repeat(X_val, 3, -1)

Y_val = [str(s) for s in val_labels]
label_encoder = LabelEncoder()
Y_val = label_encoder.fit_transform(Y_val)
Y_val_hot = to_categorical(Y_val)

# check data shape
print(Y_train.shape)
print(Y_val.shape)

print(len(np.unique(Y_train)))
print(len(np.unique(Y_val)))


(8000,)
(1200,)
20
20


# 2) Model Optimization #

## a) Keras Classifier ##

In [9]:
# parameters for tuning
epx = 50
batch_size = 8
RSCV_opts = ['adam', 'rmsprop']
krnl = (3, 3)
in_shape = (128, 128, 3)
RSCV_activation = ['relu', 'tanh', 'sigmoid']
RSCV_node1 = [2, 4, 8, 16, 32]
RSCV_node2 = [2, 4, 8, 16, 32]
RSCV_drop = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6]
RSCV_bacth = [2, 4, 6, 8, 16, 32, 64, 128]

params_grid = dict(node1 = RSCV_node1,
               node2 = RSCV_node2,
               activations = RSCV_activation,
               optimizers = RSCV_opts,
               batch_size = RSCV_bacth,
               drops = RSCV_drop)

In [10]:
# Keras Classifier needs a model builder function to instantiate a new model for each new grid point
def build_CNN(optimizers, node1, activations, node2, drops):
    CNN = Sequential([
        # first layer
        Conv2D(node1, kernel_size=krnl, activation=activations, input_shape=in_shape),
        BatchNormalization(),
        MaxPooling2D(2, 2),
        # 2nd layer
        Conv2D(node2, kernel_size=krnl, activation=activations),
        BatchNormalization(),
        MaxPooling2D(2, 2),
        # output layer
        GlobalAveragePooling2D(),
        Dense(drops),
        Dense(Y_hot.shape[1], 'softmax')
    ])
    CNN.compile(optimizers,
                loss='categorical_crossentropy',
                metrics=['accuracy'])
    
    # return built model
    return CNN

In [11]:
# build Keras Classifier
RS_cnn = KC(build_fn=build_CNN,
            verbose=1,
            epochs=5,
            batch_size=batch_size)

C:\Users\RoiMinuit\AppData\Local\Temp\ipykernel_42640\1010408690.py:2: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  RS_cnn = KC(build_fn=build_CNN,


In [12]:
# establish search space and search
G.collect()
grid_search = RSCV(RS_cnn,
                   param_distributions=params_grid,
                   cv=10,
                   scoring='accuracy',
                   verbose=3)

# execute search with categorical Y, not one-hotted
grid_search.fit(X_train, Y_train)

Fitting 10 folds for each of 10 candidates, totalling 100 fits
Epoch 1/5
57/57 [==============================] - 6s 32ms/step - loss: 2.9955 - accuracy: 0.0490
Epoch 2/5
57/57 [==============================] - 2s 31ms/step - loss: 2.9947 - accuracy: 0.0531
Epoch 3/5
57/57 [==============================] - 2s 33ms/step - loss: 2.9942 - accuracy: 0.0500
Epoch 4/5
57/57 [==============================] - 2s 33ms/step - loss: 2.9938 - accuracy: 0.0522
Epoch 5/5
25/25 [==============================] - 0s 3ms/step
[CV 1/10] END activations=tanh, batch_size=128, drops=0.5, node1=2, node2=16, optimizers=rmsprop;, score=0.056 total time=  17.4s
Epoch 1/5
[CV 2/10] END activations=tanh, batch_size=128, drops=0.5, node1=2, node2=16, optimizers=rmsprop;, score=nan total time=  14.2s
[CV 3/10] END activations=tanh, batch_size=128, drops=0.5, node1=2, node2=16, optimizers=rmsprop;, score=nan total time=  11.6s
[CV 4/10] END activations=tanh, batch_size=128, drops=0.5, node1=2, node2=16, optimize

KeyboardInterrupt: 

## b) Keras Tuner ##

In [7]:
# variables for tuning
activations = ['relu', 'leaky_relu', 'swish', 'elu']
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
checkpoint = ModelCheckpoint(model_path + '/ImgNet_Model.h5', monitor='val_loss', save_best_only=True, save_freq='epoch')

In [8]:
# base model for kt
mixed_precision.set_global_policy('mixed_float16')
def build_KT_CNN(hp):
    layers = [
        # input layer
        Conv2D(hp.Int('filters', min_value=32, max_value=128, step=32),
            input_shape=(128, 128, 3),
            kernel_size=(3, 3),
            activation=hp.Choice('activation', activations)),
            Dropout(hp.Float('drop_rate', 0.1, 0.6))
    ]
    
    # hidden layer(s)
    for l in range(hp.Int('num_layers', 1, 5)):
        layers.append(Conv2D(hp.Int('filters', min_value=32, max_value=128, step=32),
                            kernel_size=(3, 3),
                            activation=hp.Choice('activation', activations)))
    
    # output layer
    layers.append(GlobalAveragePooling2D())
    layers.append(Dropout(hp.Float('drop_rate', 0.1, 0.6)))
    layers.append(Dense(Y_hot.shape[1], activation=hp.Choice('activation', activations)))
    
    CNN = Sequential(layers)
    lr = hp.Float('learning_rate', min_value=1e-3, max_value=1e-2, sampling='LOG', default=1e-3)
    CNN.compile(optimizer=Adam(learning_rate=lr),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
    # return built model
    return CNN

build_KT_CNN(kt.HyperParameters())


INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 3050 Laptop GPU, compute capability 8.6


In [ ]:
# eastablish search space and search
tuner = kt.RandomSearch(
    hypermodel=build_KT_CNN,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    overwrite=True,
    directory=kt_path,
    project_name='ImgNetOptimization'
)

# search

G.collect()
tuner.search(X_train, Y_train,
             batch_size=20,
             epochs=100,
             validation_data=(X_val, Y_val),
             callbacks=[early_stop, checkpoint])
tuner.results_summary()

Trial 3 Complete [00h 11m 48s]
val_accuracy: 0.061666667461395264

Best val_accuracy So Far: 0.06388888880610466
Total elapsed time: 00h 36m 15s

Search: Running Trial #4

Value             |Best Value So Far |Hyperparameter
96                |128               |filters
leaky_relu        |elu               |activation
0.18265           |0.30209           |drop_rate
2                 |4                 |num_layers
0.0015723         |0.0079715         |learning_rate

Epoch 1/100
400/400 [==============================] - 28s 70ms/step - loss: 3.3317 - accuracy: 0.0486 - val_loss: 2.9961 - val_accuracy: 0.0550
Epoch 2/100
400/400 [==============================] - 27s 69ms/step - loss: 3.0112 - accuracy: 0.0476 - val_loss: 2.9961 - val_accuracy: 0.0550
Epoch 3/100
400/400 [==============================] - 27s 69ms/step - loss: 2.9961 - accuracy: 0.0520 - val_loss: 2.9961 - val_accuracy: 0.0550
Epoch 4/100
400/400 [==============================] - 27s 69ms/step - loss: 2.9974 - accuracy:

In [ ]:
# view optimized model
model = tuner.get_best_models()[0]
model.summary()
model.save("KT2.h5")

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 96)      2688      
                                                                 
 dropout (Dropout)           (None, 126, 126, 96)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 124, 124, 96)      83040     
                                                                 
 conv2d_2 (Conv2D)           (None, 122, 122, 96)      83040     
                                                                 
 global_average_pooling2d (G  (None, 96)               0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout_1 (Dropout)         (None, 96)                0         
                                                        

In [21]:
K.clear_session()
G.collect()

12159

# Don't Run #

## c) Bayesian Optimizer ##

In [18]:
# variables for tuning
activations = ['relu', 'leaky_relu', 'swish', 'elu']
early_stop = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
checkpoint = ModelCheckpoint(model_path + '/ImgNet_Model.h5', monitor='val_loss', save_best_only=True, save_freq='epoch')

In [19]:
# base model for Bayesian
def best_modHP(hp):
    layers = [
        # input layer
        Conv2D(hp.Int('filters', min_value=8, max_value=128, step=8),
            input_shape=(128, 128, 3),
            kernel_size=(3, 3),
            activation=hp.Choice('activation', activations)),
            Dropout(hp.Float('drop_rate', 0.1, 0.6))
    ]
    
    # hidden layer(s)
    for l in range(hp.Int('num_layers', 1, 5)):
        layers.append(Conv2D(hp.Int('filters', min_value=8, max_value=128, step=8),
                            kernel_size=(3, 3),
                            activation=hp.Choice('activation', activations))
                            )
        layers.append(BatchNormalization())
        layers.append(MaxPooling2D(2, 2))
        
    
    # output layer
    layers.append(GlobalAveragePooling2D())
    layers.append(Dropout(hp.Float('drop_rate', 0.1, 0.6)))
    layers.append(Dense(Y_hot.shape[1], activation=hp.Choice('activation', activations)))
    
    CNN = Sequential(layers)
    lr = hp.Float('learning_rate', min_value=1e-3, max_value=1e-2, sampling='LOG', default=1e-3)
    CNN.compile(optimizer=Adam(learning_rate=lr),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
    # return built model
    return CNN

In [20]:
# define search space and search
tuner_bay = BayesianOptimization(best_modHP,
                                 objective='val_acc',
                                 max_trials=5,
                                 executions_per_trial=1,
                                 project_name='Bayesian',
                                 overwrite=True)

# search
tuner_bay.search(X_train, Y_train,
                 epochs=10,
                 validation_data=(X_val, Y_val),
                 callbacks=[early_stop, checkpoint])

Trial 2 Complete [00h 01m 13s]

Best val_acc So Far: None
Total elapsed time: 00h 05m 20s

Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
40                |72                |filters
leaky_relu        |relu              |activation
0.18518           |0.48562           |drop_rate
4                 |5                 |num_layers
0.0085962         |0.0022459         |learning_rate

Epoch 1/10
250/250 [==============================] - 16s 56ms/step - loss: 5.3711 - accuracy: 0.0584 - val_loss: 3.9561 - val_accuracy: 0.0500
Epoch 2/10
250/250 [==============================] - 14s 54ms/step - loss: 4.1229 - accuracy: 0.0582 - val_loss: 3.1131 - val_accuracy: 0.0642
Epoch 3/10
250/250 [==============================] - 13s 54ms/step - loss: 3.0453 - accuracy: 0.0576 - val_loss: 3.0025 - val_accuracy: 0.0667
Epoch 4/10
250/250 [==============================] - 14s 54ms/step - loss: 3.0145 - accuracy: 0.0565 - val_loss: 2.9961 - val_accuracy: 0.0508
Epoch 5/1

Traceback (most recent call last):
  File "c:\Users\RoiMinuit\miniconda3\envs\tf\lib\site-packages\keras_tuner\src\engine\base_tuner.py", line 274, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "c:\Users\RoiMinuit\miniconda3\envs\tf\lib\site-packages\keras_tuner\src\engine\base_tuner.py", line 265, in _run_and_update_trial
    tuner_utils.convert_to_metrics_dict(
  File "c:\Users\RoiMinuit\miniconda3\envs\tf\lib\site-packages\keras_tuner\src\engine\tuner_utils.py", line 132, in convert_to_metrics_dict
    [convert_to_metrics_dict(elem, objective) for elem in results]
  File "c:\Users\RoiMinuit\miniconda3\envs\tf\lib\site-packages\keras_tuner\src\engine\tuner_utils.py", line 132, in <listcomp>
    [convert_to_metrics_dict(elem, objective) for elem in results]
  File "c:\Users\RoiMinuit\miniconda3\envs\tf\lib\site-packages\keras_tuner\src\engine\tuner_utils.py", line 145, in convert_to_metrics_dict
    best_value, _ = _get_best_value_a

RuntimeError: Number of consecutive failures exceeded the limit of 3.
Traceback (most recent call last):
  File "c:\Users\RoiMinuit\miniconda3\envs\tf\lib\site-packages\keras_tuner\src\engine\base_tuner.py", line 274, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "c:\Users\RoiMinuit\miniconda3\envs\tf\lib\site-packages\keras_tuner\src\engine\base_tuner.py", line 265, in _run_and_update_trial
    tuner_utils.convert_to_metrics_dict(
  File "c:\Users\RoiMinuit\miniconda3\envs\tf\lib\site-packages\keras_tuner\src\engine\tuner_utils.py", line 132, in convert_to_metrics_dict
    [convert_to_metrics_dict(elem, objective) for elem in results]
  File "c:\Users\RoiMinuit\miniconda3\envs\tf\lib\site-packages\keras_tuner\src\engine\tuner_utils.py", line 132, in <listcomp>
    [convert_to_metrics_dict(elem, objective) for elem in results]
  File "c:\Users\RoiMinuit\miniconda3\envs\tf\lib\site-packages\keras_tuner\src\engine\tuner_utils.py", line

In [ ]:
# clear gpu
K.clear_session()
G.collect()

1248